In [2]:
# Import libraries
import openai

In [3]:
# Setup OpenAI API
secret_api_key="sk-UgVTxJkXAOwlfRIJbh5t69Vl9O6ifNLXXJR941F5"
openai.api_key = secret_api_key

In [4]:
# Test
response = openai.Completion.create(engine="davinci", prompt="This is a test", max_tokens=5)

In [7]:
response

<OpenAIObject text_completion id=cmpl-2ujuJGbgpB08DdMtYiLojQhdCxlcx at 0x7fdd152a7c50> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " product with some nature sounds"
    }
  ],
  "created": 1619875511,
  "id": "cmpl-2ujuJGbgpB08DdMtYiLojQhdCxlcx",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}